In [1]:
import re
import json
import pandas as pd
import pickle
import jieba
from collections import defaultdict, Counter

In [2]:
with open('tang.pkl', 'rb') as f:
    df = pickle.load(f)
with open('poet_meta.pkl', 'rb') as f:
    author_meta = pickle.load(f)

In [3]:
def process(s):
    sentence = s.replace('。', '').replace('，', '')
    return ' '.join(jieba.cut(sentence, HMM=False))
df['raw'] = df['poem'].apply(lambda s: process(s))
df['count'] = df['author'].apply(lambda a: author_meta[a])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\cv\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built successfully.


In [4]:
wordlist = []
for v in df.values:
    wordlist += v[3].split(' ')
counter = Counter(wordlist)
print(counter.most_common(100))

[('不', 13250), ('人', 10025), ('无', 9991), ('有', 9199), ('来', 8897), ('为', 8333), ('云', 7499), ('中', 7472), ('一', 7235), ('君', 7096), ('在', 6984), ('时', 6790), ('日', 6591), ('上', 6520), ('去', 6443), ('月', 6325), ('春', 6125), ('归', 5972), ('花', 5718), ('见', 5701), ('风', 5588), ('自', 5557), ('山', 5534), ('如', 5504), ('多', 5459), ('空', 5226), ('与', 5175), ('是', 5146), ('我', 5117), ('未', 5045), ('得', 5017), ('此', 4990), ('长', 4937), ('客', 4833), ('生', 4830), ('尽', 4819), ('欲', 4723), ('何', 4717), ('秋', 4675), ('心', 4668), ('已', 4665), ('下', 4582), ('还', 4576), ('寒', 4494), ('清', 4436), ('远', 4392), ('应', 4358), ('水', 4328), ('知', 4307), ('高', 4279), ('谁', 4237), ('入', 4229), ('更', 4205), ('玉', 4127), ('行', 4114), ('天', 4109), ('闻', 4009), ('将', 4006), ('相', 3982), ('向', 3943), ('出', 3886), ('开', 3873), ('飞', 3828), ('道', 3824), ('看', 3743), ('树', 3703), ('新', 3682), ('别', 3675), ('愁', 3667), ('复', 3579), ('似', 3513), ('深', 3507), ('路', 3503), ('夜', 3498), ('朝', 3441), ('闲', 3441), ('年', 336

In [5]:
N = 500
two = {x: counter[x] for x in counter if len(x)>1}
sorted_two = sorted(two.items(), key=lambda kv: kv[1], reverse=True)[:N]
sorted_two

[('何处', 1637),
 ('不知', 1363),
 ('万里', 1291),
 ('今日', 1149),
 ('春风', 1113),
 ('白云', 1063),
 ('千里', 1039),
 ('不可', 985),
 ('不见', 883),
 ('人间', 851),
 ('无人', 839),
 ('明月', 788),
 ('不得', 787),
 ('惆怅', 778),
 ('秋风', 740),
 ('悠悠', 734),
 ('故人', 734),
 ('如何', 688),
 ('相思', 675),
 ('长安', 659),
 ('青山', 648),
 ('白日', 644),
 ('何人', 639),
 ('相逢', 615),
 ('寂寞', 592),
 ('平生', 572),
 ('少年', 571),
 ('黄金', 566),
 ('何事', 558),
 ('可怜', 535),
 ('天子', 529),
 ('回首', 523),
 ('如此', 511),
 ('今朝', 508),
 ('从此', 505),
 ('天地', 498),
 ('主人', 494),
 ('不能', 481),
 ('流水', 480),
 ('此时', 477),
 ('落日', 472),
 ('不如', 472),
 ('归来', 472),
 ('日月', 469),
 ('芳草', 467),
 ('相见', 467),
 ('何时', 466),
 ('夕阳', 466),
 ('当时', 460),
 ('天下', 453),
 ('白发', 453),
 ('日暮', 451),
 ('风雨', 444),
 ('东风', 437),
 ('将军', 436),
 ('行人', 435),
 ('参差', 434),
 ('别离', 434),
 ('十年', 421),
 ('烟霞', 417),
 ('不是', 417),
 ('洞庭', 415),
 ('归去', 413),
 ('江南', 411),
 ('唯有', 408),
 ('风吹', 407),
 ('天涯', 406),
 ('杨柳', 400),
 ('清风', 398),
 ('青云', 397),
 ('落花', 392),

In [6]:
with open('tmp_two_word.pkl', 'wb') as f:
    pickle.dump(sorted_two, f)

In [55]:
def getOrganizedDataFromTXT():
    poems = []
    with open ('./data/全唐诗.txt', 'r',encoding='utf-8') as f:
        lines = f.readlines()
        poem = ""
        author = ""
        title = ""
        for lid, line in enumerate(lines):
            if line=='\n' or line == ' \n' or line == ' \u3000\n' or line == ' \u3000 \n':
                continue
            if line[0]==' ': # find title
                authorList = re.findall(r'】\S*', line)
                titleList = re.findall(r'【\S*】', line)
                if len(authorList)==0 or len(titleList)==0:
                    continue
                poems.append({
                    'title': title,
                    'author': author,
                    'poem': poem
                })
                author = authorList[0].replace('】', '')
                title = titleList[0].replace('】', '').replace('【', '')       
                poem = ""
                continue
            poem += line.replace('\n', '')    
        poems.pop(0)
    df = pd.DataFrame(poems)
    return df
df = getOrganizedDataFromTXT()

In [136]:
def getMetaData(df):
    df['count'] = 1
    authors = df[['author','count' ]].groupby('author').count().sort_values('count', ascending=False)
    author_dicts = authors.to_dict()
    author_meta = author_dicts['count']
    author_meta[''] = 1
    return author_meta
author_meta = getMetaData(df)
with open('poet_meta.pkl', 'wb') as f:
    pickle.dump(author_meta, f)

In [141]:
df.values

array([['帝京篇十首', '李世民',
        '秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。',
        '秦川 雄 帝 宅 函 谷 壮 皇 居 绮 殿 千 寻 起 离宫 百 雉 馀 连 薨 遥 接 汉 飞 观 迥 凌 虚云 日 隐 层 阙 风烟 出 绮 疏 岩 廊 罢 机务 崇文 聊 驻 辇 玉 匣 启 龙 图 金 绳 披 凤 篆 韦 编 断 仍 续 缥 帙 舒 还 卷 对此 乃 淹留 欹 案 观 坟典 移步 出 词 林 停 舆 欣 武 宴 雕弓 写明 月 骏马 疑 流电 惊 雁 落 虚 弦 啼猿 悲 急 箭 阅 赏 诚 多 美 于 兹 乃 忘 倦 鸣 笳 临 乐 馆 眺 听 欢 芳 节 急 管 韵 朱 弦 清歌 凝 白雪 彩 凤 肃 来 仪 玄 鹤 纷 成 列 去 兹 郑 卫 声 雅音 方可 悦 芳辰 追 逸趣 禁 苑 信 多 奇 桥形 通 汉 上峰 势 接 云 危 烟霞 交 隐映 花鸟 自 参差 何如 肆 辙 迹 万里 赏 瑶池 飞 盖 去 芳 园 兰 桡 游 翠 渚 萍 间 日 彩 乱 荷 处 香风 举 桂 楫 满 中川 弦歌 振 长 屿 

In [131]:
sorted_x

[('何处', 1637),
 ('不知', 1363),
 ('万里', 1291),
 ('今日', 1149),
 ('春风', 1113),
 ('白云', 1063),
 ('千里', 1039),
 ('不可', 985),
 ('不见', 883),
 ('人间', 851),
 ('无人', 839),
 ('明月', 788),
 ('不得', 787),
 ('惆怅', 778),
 ('秋风', 740),
 ('悠悠', 734),
 ('故人', 734),
 ('如何', 688),
 ('相思', 675),
 ('长安', 659),
 ('青山', 648),
 ('白日', 644),
 ('何人', 639),
 ('相逢', 615),
 ('寂寞', 592),
 ('平生', 572),
 ('少年', 571),
 ('黄金', 566),
 ('何事', 558),
 ('可怜', 535),
 ('天子', 529),
 ('回首', 523),
 ('如此', 511),
 ('今朝', 508),
 ('从此', 505),
 ('天地', 498),
 ('主人', 494),
 ('不能', 481),
 ('流水', 480),
 ('此时', 477),
 ('落日', 472),
 ('不如', 472),
 ('归来', 472),
 ('日月', 469),
 ('芳草', 467),
 ('相见', 467),
 ('何时', 466),
 ('夕阳', 466),
 ('当时', 460),
 ('天下', 453),
 ('白发', 453),
 ('日暮', 451),
 ('风雨', 444),
 ('东风', 437),
 ('将军', 436),
 ('行人', 435),
 ('参差', 434),
 ('别离', 434),
 ('十年', 421),
 ('烟霞', 417),
 ('不是', 417),
 ('洞庭', 415),
 ('归去', 413),
 ('江南', 411),
 ('唯有', 408),
 ('风吹', 407),
 ('天涯', 406),
 ('杨柳', 400),
 ('清风', 398),
 ('青云', 397),
 ('落花', 392),

In [99]:
import jieba

In [104]:
sentence = poems[120]['poem'].replace('。', '').replace('，', '')
seperated = ' '.join(jieba.cut(sentence, HMM=False))
seperated

'桂 殿 与 山 连 兰 汤 涌 自然 阴 崖 含 秀色 温 谷 吐 潺 湲 绩 为 蠲 邪 著 功 因 养 正 宣 愿 言 将 亿兆 同 此 共 昌 延'

In [105]:
' '.join(jieba.cut(sentence, HMM=True))

'桂殿 与 山连兰 汤 涌 自然 阴崖 含 秀色 温谷 吐潺 湲 绩为 蠲 邪 著功 因养 正 宣愿言 将 亿兆 同此 共昌延'